In [1072]:
import sim 
import simConst 
import numpy as np 
import sympy as sp  

In [1073]:
def connect(port): 
    sim.simxFinish(-1) #Desconecta todos los puertos. 
    clientID = sim.simxStart('127.0.0.1', port, True, True, 2000, 5)
    if clientID == 0: 
        print("Conectado a : ", port)
    else: 
        print("No se pudo conectar a: ", port)
    return clientID 
clientID = connect(1111)

Conectado a :  1111


In [1074]:
ret, joint1_handle = sim.simxGetObjectHandle(clientID, "joint1", sim.simx_opmode_blocking)
ret, joint2_handle = sim.simxGetObjectHandle(clientID, "joint2", sim.simx_opmode_blocking)
ret, joint3_handle = sim.simxGetObjectHandle(clientID, "joint3", sim.simx_opmode_blocking)
ret, pos_joint0 = sim.simxGetJointPosition(clientID, joint1_handle, sim.simx_opmode_blocking)
ret, pos_joint1 = sim.simxGetJointPosition(clientID, joint2_handle, sim.simx_opmode_blocking)
ret, pos_joint3 = sim.simxGetJointPosition(clientID, joint3_handle, sim.simx_opmode_blocking)
print(f"Posicion Joint1: {pos_joint0}, Posicion Joint2: {pos_joint1}")

Posicion Joint1: 0.7853958606719971, Posicion Joint2: -0.07783675193786621


In [1075]:
## Longitud de los eslabones: 
d1 = 0.1
d2 = 0.1
d3 = 0.1

q1 = np.radians(0)
q2 = np.radians(0)
q3 = np.radians(0)
ret = sim.simxSetJointTargetPosition(clientID, joint1_handle, q1, sim.simx_opmode_blocking)
ret = sim.simxSetJointTargetPosition(clientID, joint2_handle, q2, sim.simx_opmode_blocking)
ret = sim.simxSetJointTargetPosition(clientID, joint3_handle, q3, sim.simx_opmode_blocking)
#Cinematica directa:
# Si conozco las posiciones de los angulos puedo conocer
# la pos final del robot.  
# x = d3*np.sin(q0)*np.sin(q1)
# y = -d3*np.cos(q0)*np.sin(q1)
# z = d1+d2+d3*np.cos(q1)
# print(f"Pos con cinematica en x: {x}, pos en y: {y}, pos en z: {z}") 
retrunCode, pos_final_handle = sim.simxGetObjectHandle(clientID, "pos_final", sim.simx_opmode_blocking)
pos_final = sim.simxGetObjectPosition(clientID, pos_final_handle, -1, sim.simx_opmode_blocking)
print("Posicion del extremo final: ", pos_final)

Posicion del extremo final:  (0, [-5.885871360078454e-05, 1.1768133845180273e-07, 0.3149998188018799])


In [1076]:
#condiciones iniciales: 
p0 = {
  "x": 0, 
  "y": 0,
  "z": 0.1
}
p1 = {
  "x": 0, 
  "y": 0,
  "z": 0.2
}
p2 = {
  "x": 0, 
  "y": 0,
  "z": 0.31
}

In [1077]:
#def Fabrik IK: 
#Fabrik
def fabrik(dist_vectores, segments, tgt, base):     
    base = base 
    segments = segments 
    dist_vectores = dist_vectores
    p_prima = [tgt]
    
    for i in range(len(segments) - 1): 
        eje_x = segments[i+1]["x"] - p_prima[i]["x"] 
        eje_y = segments[i+1]["y"] - p_prima[i]["y"]
        eje_z = segments[i+1]["z"] - p_prima[i]["z"]
        ux = eje_x/(np.sqrt(eje_x**2 + eje_y**2 + eje_z**2))
        uy = eje_y/(np.sqrt(eje_x**2 + eje_y**2 + eje_z**2))
        uz = eje_z/(np.sqrt(eje_x**2 + eje_y**2 + eje_z**2))
        
        prima_x = p_prima[i]["x"] + dist_vectores[i]*ux
        prima_y = p_prima[i]["y"] + dist_vectores[i]*uy
        prima_z = p_prima[i]["z"] + dist_vectores[i]*uz
        prima = {
            "x": prima_x, 
            "y": prima_y, 
            "z": prima_z
        }
        p_prima.append(prima)

    #Backward:
    # [d1 ,d2]
    dist_vectores.reverse()
    print("p_prima:", len(p_prima))
    p_prima.reverse()
    p_2prima = [base] 
    print("Distancia de vectores: ", dist_vectores)

    for i in range(len(segments) - 1): 
        eje_x = p_prima[i+1]["x"] - p_2prima[i]["x"] 
        eje_y = p_prima[i+1]["y"] - p_2prima[i]["y"]
        eje_z = p_prima[i+1]["z"] - p_2prima[i]["z"]
        ux = eje_x/(np.sqrt(eje_x**2 + eje_y**2 + eje_z**2))
        uy = eje_y/(np.sqrt(eje_x**2 + eje_y**2 + eje_z**2))
        uz = eje_z/(np.sqrt(eje_x**2 + eje_y**2 + eje_z**2))
        
        prima_x = p_2prima[i]["x"] + dist_vectores[i]*ux
        prima_y = p_2prima[i]["y"] + dist_vectores[i]*uy
        prima_z = p_2prima[i]["z"] + dist_vectores[i]*uz
        prima = {
            "x": prima_x, 
            "y": prima_y,
            "z": prima_z
        }
        p_2prima.append(prima)

    print("p_2prima:", p_2prima)
    
    return p_2prima   

In [1078]:
#Main Fabrik
retrunCode, target_handle = sim.simxGetObjectHandle(clientID, "target", sim.simx_opmode_blocking)
returnCode, pos_target = sim.simxGetObjectPosition(clientID, target_handle, -1, sim.simx_opmode_blocking)
d1 = 0.1
d2 = 0.11
print("Target: ", pos_target)
tgt = {
  "x": pos_target[0], 
  "y": pos_target[1],
  "z": pos_target[2] 
}
#tgt para hacer debug: z= 0.31
# tgt = {
#   "x": 0, 
#   "y": 0,
#   "z": 0.31 
# }

dist_vectores = [d2, d1]
segments = [p2, p1, p0] 
base = segments[len(segments) - 1]

for i in range(2): 
  p_2prima = fabrik(dist_vectores, segments, tgt, base)
  segments = p_2prima 
  dist_vectores = [d2, d1]
  print("Segments news: ", segments)

Target:  [0.07499998062849045, 0.05000012367963791, 0.2500000596046448]
p_prima: 3
Distancia de vectores:  [0.1, 0.11]
p_2prima: [{'x': 0, 'y': 0, 'z': 0.1}, {'x': -0.005201501596091481, 'y': -0.0034676772039806654, 'z': 0.19980440669607158}, {'x': 0.07597633978829986, 'y': 0.05065103156437506, 'z': 0.25061113291826087}]
Segments news:  [{'x': 0, 'y': 0, 'z': 0.1}, {'x': -0.005201501596091481, 'y': -0.0034676772039806654, 'z': 0.19980440669607158}, {'x': 0.07597633978829986, 'y': 0.05065103156437506, 'z': 0.25061113291826087}]
p_prima: 3
Distancia de vectores:  [0.1, 0.11]
p_2prima: [{'x': 0, 'y': 0, 'z': 0.1}, {'x': -0.006210726031965096, 'y': -0.004140495332616622, 'z': 0.1997210267724739}, {'x': 0.07515355220809468, 'y': 0.05010250501240018, 'z': 0.25009513857650123}]
Segments news:  [{'x': 0, 'y': 0, 'z': 0.1}, {'x': -0.006210726031965096, 'y': -0.004140495332616622, 'z': 0.1997210267724739}, {'x': 0.07515355220809468, 'y': 0.05010250501240018, 'z': 0.25009513857650123}]


In [1079]:
##angulos FABRIK 
# def angulos_fabrik(p_2prima): 
#     print(f"target: {tgt} \n")
#     print("p_2prima", p_2prima)
#     x = p_2prima[1]["x"]
#     y = p_2prima[1]["y"]
#     z = p_2prima[1]["z"]
#     print(f"p_prima 2do punto: x:{x}, y:{y}, z: {z}")
    
#     #Primer eslabon: 
#     q1 = np.arctan2(y,x)
#     eje_x1 = 0
#     eje_y1 = 0
#     eje_z1 = d1 

#     #Cinematica directa segundo eslabon:
#     success = 0 
#     for q2 in np.arange(0, np.pi*2, 0.0001): 
#         if (abs(d1 + d2*np.cos(q2) - z) < 0.001): 
#             print("Se encontro un angulo ")
#             success = 1
#             break
#     if (success == 1): 
#         print(f"Se encontro el angulo: {q2}")
#     else: 
#         print("No se encontro un angulo")    
#     eje_x2 = -d2*np.cos(q1)*np.sin(q2) 
#     eje_y2 = -d2*np.sin(q1)*np.sin(q2)
#     eje_z2 = d1 + d2*np.cos(q2)
#     print(f"eje x: {eje_x2}, eje y: {eje_y2}, eje z: {eje_z2} \n")

#     #Cinematica directa tercer eslabon:
#     x = p_2prima[2]["x"]
#     y = p_2prima[2]["y"]
#     z = p_2prima[2]["z"]
#     print(f"p_prima 3er punto: x:{x}, y:{y}, z: {z}")
#     success = 0  
#     for q3 in np.arange(0, np.pi*2, 0.0001): 
#         if (abs(d1 + d3*np.cos(q2 + q3) + d2*np.cos(q2) - z) < 0.001): 
#             print("Se encontro un angulo ")
#             success = 1
#             break
#     eje_y3 = np.sin(q1)*(d3*np.sin(q2 + q3) + d2*np.sin(q2))
#     eje_z3 = d1 + d3*np.cos(q2 + q3) + d2*np.cos(q2)
#     print(f"eje y eslabon3: {eje_y3}, eje z eslabon3: {eje_z3}")
    
#     if (success == 1): 
#         print(f"Se encontro el angulo: {q2}")
#     else: 
#         print("No se encontro un angulo")
#     print(f"q1: {q1}, q2: {q2}, q3: {q2}")
#     return [q1, q2, q3]

In [1080]:
## Calculo de los angulos mediante CCD IK
def ccd_ik(home_points ,p_2prima): 
    print(f"target: {tgt} \n")
    print("p_2prima", p_2prima, "home points", home_points)
    x = p_2prima[1]["x"]
    y = p_2prima[1]["y"]
    z = p_2prima[1]["z"]
    
    #Primer eslabon: 
    q1 = np.arctan2(y,x)
    print("q1: ", q1)
    if q1 < 0: 
        q1 = -1*(-np.pi - q1)
        print(q1)
    angles = [q1]
    
    for i in range(len(p_2prima) - 1): 
        #U1 Vector que apunta hacia las p_prima
        eje1_x = p_2prima[i+1]["x"] - home_points[i]["x"]
        eje1_y = p_2prima[i+1]["y"] - home_points[i]["y"]
        eje1_z = p_2prima[i+1]["z"] - home_points[i]["z"]
        u1_x = eje1_x/(np.sqrt(eje1_x**2 + eje1_y**2 + eje1_z**2))
        u1_y = eje1_y/(np.sqrt(eje1_x**2 + eje1_y**2 + eje1_z**2))
        u1_z = eje1_z/(np.sqrt(eje1_x**2 + eje1_y**2 + eje1_z**2))
        #U2
        eje2_x = home_points[i+1]["x"] - home_points[i]["x"]
        eje2_y = home_points[i+1]["y"] - home_points[i]["y"]
        eje2_z = home_points[i+1]["z"] - home_points[i]["z"]
        #Modulo
        u2_x = eje2_x/(np.sqrt(eje2_x**2 + eje2_y**2 + eje2_z**2))
        u2_y = eje2_y/(np.sqrt(eje2_x**2 + eje2_y**2 + eje2_z**2))
        u2_z = eje2_z/(np.sqrt(eje2_x**2 + eje2_y**2 + eje2_z**2))
        u1 = np.array([u1_x, u1_y, u1_z])
        u2 = np.array([u2_x, u2_y, u2_z])
        #Calculo del signo del angulo: 
        sign = np.sign(np.cross(u1, u2)) 
        print("Cross product: ", np.cross(u1,u2) ,"Signo del angulo: ", sign, "\n")
        dot_product = np.dot(u1, u2) 
        print("Dot product: ", dot_product)
        #El -1 es por cuestiones del simulador que aun no entiendo: 
        angle = sign * np.arccos(dot_product)
        angles.append(angle)

        if i == 0:  
            home_points[i+1]["x"] = p_2prima[i+1]["x"]
            home_points[i+1]["y"] = p_2prima[i+1]["y"]
            home_points[i+1]["z"] = p_2prima[i+1]["z"]
            print("New home x: ", home_points[i+1]["x"])
            print("New home y: ", home_points[i+1]["y"])
            print("New home z: ", home_points[i+1]["z"])

            #Actualizamos el punto siguiente con CD:
            q2 = angles[1][0]
            d = d2 + d3
            home_points[i+2]["x"] = d*np.cos(q1)*np.sin(q2)
            home_points[i+2]["y"] = d*np.sin(q1)*np.sin(q2)
            home_points[i+2]["z"] = d1 + d*np.cos(q2)
            print("Ultimo punto actualizado x: ",home_points[i+2]["x"])
            print("Ultimo punto actualizado y: ",home_points[i+2]["y"])
            print("Ultimo punto actualizado z: ",home_points[i+2]["z"])
        
        
    print("Angulos calculados: ", angles) 
    q = [angles[0], angles[1][0], angles[2][0]]
    
    return q

In [1081]:
#Calculo de los angulos entre los eslabones: 
#condiciones iniciales: 
p0 = {
  "x": 0, 
  "y": 0,
  "z": 0.1
}
p1 = {
  "x": 0, 
  "y": 0,
  "z": 0.2
}
p2 = {
  "x": 0, 
  "y": 0,
  "z": 0.3
}

home_points = [p0, p1, p2]
#q = angulos_fabrik(p_2prima)
q = ccd_ik(home_points, p_2prima)

#Movimiento de los motores:  
ret = sim.simxSetJointTargetPosition(clientID, joint1_handle, q[0], sim.simx_opmode_blocking)
ret = sim.simxSetJointTargetPosition(clientID, joint2_handle, q[1], sim.simx_opmode_blocking)
ret = sim.simxSetJointTargetPosition(clientID, joint3_handle, q[2], sim.simx_opmode_blocking)

target: {'x': 0.07499998062849045, 'y': 0.05000012367963791, 'z': 0.2500000596046448} 

p_2prima [{'x': 0, 'y': 0, 'z': 0.1}, {'x': -0.006210726031965096, 'y': -0.004140495332616622, 'z': 0.1997210267724739}, {'x': 0.07515355220809468, 'y': 0.05010250501240018, 'z': 0.25009513857650123}] home points [{'x': 0, 'y': 0, 'z': 0.1}, {'x': 0, 'y': 0, 'z': 0.2}, {'x': 0, 'y': 0, 'z': 0.3}]
q1:  -2.553588789175474
0.5880038644143193
Cross product:  [-0.04140495  0.06210726  0.        ] Signo del angulo:  [-1.  1.  0.] 

Dot product:  0.9972102677247388
New home x:  -0.006210726031965096
New home y:  -0.004140495332616622
New home z:  0.1997210267724739
Ultimo punto actualizado x:  -0.013042524667126663
Ultimo punto actualizado y:  -0.008695040198494881
Ultimo punto actualizado z:  0.30941415622219515
Cross product:  [ 5.10703769e-01 -7.66053561e-01 -2.77555756e-17] Signo del angulo:  [ 1. -1. -1.] 

Dot product:  0.390312183741432
Angulos calculados:  [0.5880038644143193, array([-0.07471319,  

In [1082]:
retrunCode, pos_final_handle = sim.simxGetObjectHandle(clientID, "pos_final", sim.simx_opmode_blocking)
pos_final = sim.simxGetObjectPosition(clientID, pos_final_handle, -1, sim.simx_opmode_blocking)
print("Posicion del extremo final: ", pos_final)
retrunCode, pos_final_handle = sim.simxGetObjectHandle(clientID, "pos_dummy2", sim.simx_opmode_blocking)
eslabon1_pos = sim.simxGetObjectPosition(clientID, pos_final_handle, -1, sim.simx_opmode_blocking)
print("Posicion dummy 2: ", eslabon1_pos)

Posicion del extremo final:  (0, [0.0834176316857338, 0.05553646385669708, 0.24894079566001892])
Posicion dummy 2:  (0, [-0.007649794686585665, -0.0050921207293868065, 0.20982614159584045])
